In [ ]:
import streamlit as st
import pandas as pd
import plotly.express as px
import nltk
from langdetect import detect
from nltk.util import ngrams
import spacy
import yake
from transformers import BertTokenizer, BertModel, AutoTokenizer, AutoModelForSequenceClassification
import torch
import torch.nn.functional as F
import fasttext

# Initialize the model and tokenizer


tokenizer = AutoTokenizer.from_pretrained('bert-base-multilingual-cased')
tokenizer = AutoTokenizer.from_pretrained('bert-base-multilingual-cased')
models = {
    'en': AutoModelForSequenceClassification.from_pretrained('Worgu/Final_Project_finetuned_bert-base-multilingual-cased_english'),
    'de': AutoModelForSequenceClassification.from_pretrained('Worgu/Final_Project_finetuned_bert-base-multilingual-cased_german'),
    'fr': AutoModelForSequenceClassification.from_pretrained('Worgu/Final_Project_finetuned_bert-base-multilingual-cased_french'),
    'es': AutoModelForSequenceClassification.from_pretrained('Worgu/Final_Project_finetuned_bert-base-multilingual-cased_spanish')}


# Download NLTK stopwords and spacy model
nltk.download("stopwords")
nltk.download("punkt")
spacy.cli.download("en_core_web_sm")
nlp = spacy.load("en_core_web_sm")

# YAKE keyword extraction
yake_kw_extractor = yake.KeywordExtractor()


def sentiment_analysis(review: str, lang):
    inputs = tokenizer(review, return_tensors="pt", truncation=True, padding=True)
    model = models.get(lang, models['en'])  # Use English model as default
    outputs = model(**inputs)
    probabilities = F.softmax(outputs.logits, dim=1)
    sentiment_idx = torch.argmax(probabilities).item()

    sentiment_label = None
    if sentiment_idx == 0:
        sentiment_label = " POSITIVE"
    elif sentiment_idx == 1:
        sentiment_label = "NEGATIVE"
    elif sentiment_idx == 2:
        sentiment_label = "NEUTRAL"

    return {"label": sentiment_label, "score": probabilities[0, sentiment_idx].item()}
    

def analyze_reviews(reviews: str):
    reviews_list = reviews.split("\n")
    results = []

    for review in reviews_list:
        detected_language = detect(review)
        result = sentiment_analysis(review, detected_language)
        result["language"] = detected_language
        result["review"] = review
        results.append(result)

    return results

 
def get_ngrams(text: str, n: int = 2):
    tokens = nltk.word_tokenize(text)
    n_grams = list(ngrams(tokens, n))
    return n_grams

def get_yake_keywords(text: str, n: int = 5):
    return yake_kw_extractor.extract_keywords(text)[:n]

def emoji(sentiment_score):
    if sentiment_score >= 0.8:
        return "😄"
    elif sentiment_score >= 0.2 and sentiment_score < 0.8:
        return "😊"
    elif sentiment_score > -0.2 and sentiment_score < 0.2:
        return "😐"
    elif sentiment_score > -0.8 and sentiment_score <= -0.2:
        return "☹️"
    else:
        return "😠"


def main():
    st.title("Multilingual Review Sentiment Analysis")
    st.write("""
    Enter your text in the text area below. You can input multiple texts separated by line breaks. The sentiment analysis results for each review will be displayed after analysis.
    """)

    reviews = st.text_area("Input your Amazon reviews here:", height=200)

    if st.button("Analyze text"):
        results = analyze_reviews(reviews)
        for result in results:
            sentiment_label_text = result['label']
            sentiment_score = result['score']
            sentiment_emoji = emoji(sentiment_score)

            st.write(f"Review: {result['review']} | Language: {result['language']} | Sentiment: {sentiment_label_text} {sentiment_emoji}")

        st.subheader("N-gram Analysis")
        n_value = st.slider("Select N value for N-grams:", 2, 5, 2)
        ngrams_list = get_ngrams(reviews, n=n_value)
        st.write(f"Top {n_value}-grams:")
        st.write(ngrams_list)

        st.subheader("Keyword Extraction")
        num_keywords = st.slider("Select number of keywords to extract:", 5, 20, 5)
        yake_keywords = get_yake_keywords(reviews, n=num_keywords)
        st.write("YAKE extracted keywords:")
        st.write(yake_keywords)

        st.subheader("Summary Statistics")
        data = {
            "Sentiment Score": [result["score"] for result in results],
            "Language": [result["language"] for result in results]
        }
        df = pd.DataFrame(data)
        st.write(df.describe())

        st.subheader("Interactive Visualizations")
        fig = px.histogram(df, x="Sentiment Score", nbins=20, title="Sentiment Score Distribution")
        st.plotly_chart(fig)

        lang_count = df["Language"].value_counts()
        fig2 = px.pie(lang_count, values=lang_count.values, names=lang_count.index, title="Language Distribution")
        st.plotly_chart(fig2)


if __name__ == "__main__":
    main()